# 1. - Cкачиваем обученные модели и репозиторий

In [1]:
%cd /content
!rm -rf HW-2_env
!git clone https://github.com/TebelevGt/HW-2_env.git
%cd HW-2_env/rl-shortest-path-agent

/content
Cloning into 'HW-2_env'...
remote: Enumerating objects: 457, done.
remote: Counting objects: 100% (103/103), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 457 (delta 50), reused 89 (delta 41), pack-reused 354 (from 1)
Receiving objects: 100% (457/457), 6.85 MiB | 12.33 MiB/s, done.
Resolving deltas: 100% (216/216), done.
/content/HW-2_env/rl-shortest-path-agent


# 2. - Логика импортов

In [2]:
%%capture
import os
os.environ["UNSLOTH_VLLM_STANDBY"] = "1" # [NEW] Extra 30% context lengths!
if "COLAB_" not in "".join(os.environ.keys()):
    # If you're not in Colab, just use pip install or uv pip install
    !pip install unsloth vllm
else:
    pass # For Colab / Kaggle, we need extra instructions hidden below \/

In [3]:
#@title Colab Extra Install { display-mode: "form" }
#%%capture
import os
!pip install --upgrade -qqq uv
if "COLAB_" not in "".join(os.environ.keys()):
    # If you're not in Colab, just use pip install!
    !pip install unsloth vllm
else:
    try: import numpy, PIL; _numpy = f'numpy=={numpy.__version__}'; _pil = f'pillow=={PIL.__version__}'
    except: _numpy = "numpy"; _pil = "pillow"
    try: import subprocess; is_t4 = "Tesla T4" in str(subprocess.check_output(["nvidia-smi"]))
    except: is_t4 = False
    _vllm, _triton = ('vllm==0.9.2', 'triton==3.2.0') if is_t4 else ('vllm==0.15.1', 'triton')
    !uv pip install -qqq --upgrade {_vllm} {_numpy} {_pil} torchvision bitsandbytes xformers unsloth
    !uv pip install -qqq {_triton}
!uv pip install transformers==4.56.2
!uv pip install --no-deps trl==0.22.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.1/23.1 MB 74.3 MB/s eta 0:00:00:00:0100:01
Using Python 3.12.12 environment at: /usr
Resolved 18 packages in 78ms                                         
Prepared 1 package in 484ms                                              
Uninstalled 1 package in 76ms
Installed 1 package in 56ms                                 
 - transformers==4.57.6
 + transformers==4.56.2
Using Python 3.12.12 environment at: /usr
Resolved 1 package in 1ms                                            
Prepared 1 package in 92ms                                               
Uninstalled 1 package in 2ms
Installed 1 package in 6ms                                  
 - trl==0.24.0
 + trl==0.22.2


In [ ]:
import os
import torch
from unsloth import FastLanguageModel
from trl import GRPOTrainer, GRPOConfig
from envs import (
    get_shortest_path_dataset, 
    correctness_reward_func, 
    reasoning_length_reward_func, 
    format_reward_func
)

# 3. - Обучение

In [5]:
def train_grpo_model(
    dataset_path: str, 
    output_dir: str = "/kaggle/working/grpo_model_curiculum",
    max_steps: int = 250,
    num_generations: int = 8
):
    """
    Обучает модель Qwen2.5-1.5B методом GRPO на заданном датасете.
    """
    
    # --- 1. Конфигурация модели ---
    max_seq_length = 4096 * 2
    lora_rank = 64

    print(f"--- Загрузка модели и токенизатора ---")
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "unsloth/Qwen2.5-1.5B-Instruct",
        max_seq_length = max_seq_length,
        load_in_4bit = True,
        fast_inference = True,
        max_lora_rank = lora_rank,
        gpu_memory_utilization = 0.8,
    )

    model = FastLanguageModel.get_peft_model(
        model,
        r = lora_rank,
        target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
        lora_alpha = lora_rank * 2,
        lora_dropout = 0.05,
        use_gradient_checkpointing = "unsloth",
        random_state = 3407,
        use_rslora = True,
    )

    tokenizer.padding_side = "left"

    # --- 2. Настройка GRPO ---
    print(f"--- Настройка GRPO Trainer (Dataset: {dataset_path}) ---")
    training_args = GRPOConfig(
        use_vllm = True, 
        learning_rate = 5e-6,
        adam_beta1 = 0.9,
        adam_beta2 = 0.99,
        weight_decay = 0.1,
        warmup_ratio = 0.1,
        lr_scheduler_type = "cosine",
        optim = "adamw_8bit",
        logging_steps = 1,
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 1,
        num_generations = num_generations, 
        max_prompt_length = 256,
        max_completion_length = 200,
        max_steps = max_steps,
        save_steps = max_steps,
        max_grad_norm = 0.1,
        report_to = "none",
        output_dir = "outputs",
        shuffle_dataset = False
    )

    trainer = GRPOTrainer(
        model = model,
        processing_class = tokenizer,
        reward_funcs = [
            correctness_reward_func,
            reasoning_length_reward_func,
            format_reward_func
        ],
        args = training_args,
        train_dataset = get_shortest_path_dataset(dataset_path)
    )

    # --- 3. Обучение и сохранение ---
    print(f"--- Начало обучения ---")
    trainer.train()
    
    print(f"--- Сохранение модели в {output_dir} ---")
    trainer.save_model(output_dir)
    
    return model, tokenizer

# Пример вызова:
model, tokenizer = train_grpo_model('data/train_curriculum.pkl')

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2026-02-23 08:05:36.839407: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1771833937.073117      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1771833937.135188      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1771833937.674762      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771833937.674798      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771833937.674801      55 computation_placer.cc:177] computation placer alr

INFO 02-23 08:06:14 [__init__.py:244] Automatically detected platform cuda.
ERROR 02-23 08:06:16 [fa_utils.py:57] Cannot use FA version 2 is not supported due to FA2 is only supported on devices with compute capability >= 8
🦥 Unsloth Zoo will now patch everything to make training faster!
--- Загрузка модели и токенизатора ---
INFO 02-23 08:06:25 [vllm_utils.py:723] Unsloth: Patching vLLM v1 graph capture
INFO 02-23 08:06:25 [vllm_utils.py:752] Unsloth: Patching vLLM v0 graph capture
==((====))==  Unsloth 2026.2.1: Fast Qwen2 patching. Transformers: 4.56.2. vLLM: 0.9.2.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.563 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/qwen2.5-1.5b-instru

`torch_dtype` is deprecated! Use `dtype` instead!


WARNING 02-23 08:06:44 [config.py:3371] Casting torch.bfloat16 to torch.float16.
INFO 02-23 08:06:44 [config.py:1472] Using max model len 8192
WARNING 02-23 08:06:45 [arg_utils.py:1735] Compute Capability < 8.0 is not supported by the V1 Engine. Falling back to V0. 
INFO 02-23 08:06:47 [config.py:2285] Chunked prefill is enabled with max_num_batched_tokens=4096.
Unsloth: vLLM Bitsandbytes config using kwargs = {'load_in_8bit': False, 'load_in_4bit': True, 'bnb_4bit_compute_dtype': 'float16', 'bnb_4bit_quant_storage': 'uint8', 'bnb_4bit_quant_type': 'nf4', 'bnb_4bit_use_double_quant': True, 'llm_int8_enable_fp32_cpu_offload': False, 'llm_int8_has_fp16_weight': False, 'llm_int8_skip_modules': ['lm_head', 'multi_modal_projector', 'merger', 'modality_projection', 'model.layers.0.self_attn', 'model.layers.1.mlp', 'model.layers.2.mlp', 'model.layers.3.mlp', 'model.layers.7.mlp', 'model.layers.24.mlp', 'model.layers.26.mlp', 'model.layers.15.self_attn'], 'llm_int8_threshold': 6.0}
INFO 02-23 

...

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2026.2.1 patched 28 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


--- Настройка GRPO Trainer (Dataset: data/train_curriculum.pkl) ---
Unsloth: We now expect `per_device_train_batch_size` * `gradient_accumulation_steps` * `world_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 8
--- Начало обучения ---


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 2,000 | Num Epochs = 1 | Total steps = 250
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 1 x 1) = 8
 "-____-"     Trainable parameters = 73,859,072 of 1,617,573,376 (4.57% trained)


INFO 02-23 08:07:47 [peft_helper.py:58] Loading LoRA weights trained with rsLoRA.
Unsloth: Will smartly offload gradients to save VRAM!

[Train] Start: 1 | End: 5 | Ideal: 1, 6, 5 (Cost: 2) | Agent: Empty/Parse Error | Reward: 0.0

[Train] Start: 1 | End: 5 | Ideal: 1, 6, 5 (Cost: 2) | Agent: Empty/Parse Error | Reward: 0.0

[Train] Start: 1 | End: 5 | Ideal: 1, 6, 5 (Cost: 2) | Agent: Empty/Parse Error | Reward: 0.0

[Train] Start: 1 | End: 5 | Ideal: 1, 6, 5 (Cost: 2) | Agent: Empty/Parse Error | Reward: 0.0

[Train] Start: 1 | End: 5 | Ideal: 1, 6, 5 (Cost: 2) | Agent: Empty/Parse Error | Reward: 0.0

[Train] Start: 1 | End: 5
  Ideal Path: 1, 6, 5 (Cost: 2)
  Agent Path: [1, 4, 5]
  Agent Cost: 19
  Exists in Graph: True | Reached End: True
  Reward: 4.3000

[Train] Start: 1 | End: 5 | Ideal: 1, 6, 5 (Cost: 2) | Agent: [0, 4, 5] | Status: Wrong Start | Reward: 0.0

[Train] Start: 1 | End: 5 | Ideal: 1, 6, 5 (Cost: 2) | Agent: Empty/Parse Error | Reward: 0.0


KeyboardInterrupt: 